In [1]:
# Install dependencies
!pip install opendatasets gradio tensorflow --quiet

import opendatasets as od
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import gradio as gr
import numpy as np
import os
import shutil

# Download dataset
od.download("https://www.kaggle.com/datasets/jakupymeraj/cats-and-dogs-image-dataset")

# Copy small number of images for quick training
base_path = "/content/cats-and-dogs-image-dataset/dataset/training_set"
small_train_path = "/content/small_train"
os.makedirs(small_train_path + "/cats", exist_ok=True)
os.makedirs(small_train_path + "/dogs", exist_ok=True)

for cls in ['cats', 'dogs']:
    for i, img in enumerate(os.listdir(f"{base_path}/{cls}")):
        if i >= 100: break  # Use 100 per class for better learning
        shutil.copy(f"{base_path}/{cls}/{img}", f"{small_train_path}/{cls}")

# Image parameters
IMG_SIZE = (160, 160)
BATCH_SIZE = 16

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    small_train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    small_train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Load pre-trained MobileNetV2 without top layer
base_model = tf.keras.applications.MobileNetV2(input_shape=(*IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # Freeze base

# Add custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train (only final layer) for quick demo
model.fit(train_generator, validation_data=val_generator, epochs=3)

# Prediction function
def classify_image(img):
    img = img.resize(IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    return "Dog 🐶" if prediction > 0.5 else "Cat 🐱"

# Gradio UI
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Text(label="Prediction"),
    title="Cat vs Dog Classifier (MobileNetV2 Transfer Learning)",
    description="Upload a cat or dog image. Trained on 200 images for quick and accurate prediction."
)

interface.launch()

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rubabshahzadi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jakupymeraj/cats-and-dogs-image-dataset


100%|██████████| 217M/217M [00:00<00:00, 503MB/s]



Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 629ms/step - accuracy: 0.6947 - loss: 0.5711 - val_accuracy: 0.8500 - val_loss: 0.4084
Epoch 2/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 445ms/step - accuracy: 0.9039 - loss: 0.2782 - val_accuracy: 0.9000 - val_loss: 0.2621
Epoch 3/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 532ms/step - accuracy: 0.9281 - loss: 0.2168 - val_accuracy: 0.9250 - val_loss: 0.2089
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc791a971cff538507.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
